In [1]:
import itertools
import numpy as np
import pandas as pd
import sys
import os
from datetime import timedelta

from preprocess import read_and_preprocess_data

In [2]:
in_file = "challenge/data/device_activations_small.csv"
BATCH_SIZE = 16

In [3]:
feature_batch, label_batch, device_list = read_and_preprocess_data(in_file, batch_size=BATCH_SIZE)
print(feature_batch[0])
print(feature_batch[1])
print(feature_batch[-1])
print("Feature batch: ", feature_batch.shape)
print("Label batch: ", label_batch.shape)

File challenge/data/device_activations_small.csv has 125 timesteps (hours)
initial features shape:  (125, 8)
Full sequence length:  60
Sequence 0 has start index 0 and end index 60
(60, 8)
Sequence 1 has start index 4 and end index 64
(60, 8)
Sequence 2 has start index 8 and end index 68
(60, 8)
Sequence 3 has start index 12 and end index 72
(60, 8)
Sequence 4 has start index 16 and end index 76
(60, 8)
Sequence 5 has start index 20 and end index 80
(60, 8)
Sequence 6 has start index 24 and end index 84
(60, 8)
Sequence 7 has start index 28 and end index 88
(60, 8)
Sequence 8 has start index 32 and end index 92
(60, 8)
Sequence 9 has start index 36 and end index 96
(60, 8)
Sequence 10 has start index 40 and end index 100
(60, 8)
Sequence 11 has start index 44 and end index 104
(60, 8)
Sequence 12 has start index 48 and end index 108
(60, 8)
Sequence 13 has start index 52 and end index 112
(60, 8)
Sequence 14 has start index 56 and end index 116
(60, 8)
Sequence 15 has start index 60 an

In [4]:
train_ratio = 0.9
train_len = int(train_ratio * len(feature_batch) // BATCH_SIZE) * BATCH_SIZE
print("Train len: ", train_len)

train_feature_batch = feature_batch[:train_len]
test_feature_batch = feature_batch[train_len:]
train_label_batch = label_batch[:train_len]
test_label_batch = label_batch[train_len:]

print(train_feature_batch.shape)
print(test_feature_batch.shape)

Train len:  32
(32, 20, 8)
(16, 20, 8)


In [5]:
def calc_ratio_positive_outputs_per_device(labels):
    ratio_per_device = np.sum(labels, axis=0) / labels.shape[0]
    print("Percentage of positive outputs per device: ", ratio_per_device)
    return np.array(ratio_per_device)
ratio_positive_outputs_per_device = calc_ratio_positive_outputs_per_device(label_batch.reshape([-1, label_batch.shape[-1]]))

Percentage of positive outputs per device:  [0.16979167 0.1875     0.11770833 0.10104167 0.13645833 0.046875  ]


In [6]:
import matplotlib.pyplot as plt
import math
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM
from keras import objectives
from keras import backend as K

from sklearn.metrics import mean_squared_error
from math import sqrt

from bayes_opt import BayesianOptimization

Using TensorFlow backend.


In [7]:
USE_WEIGHTED_LOSS = True

In [8]:
# Create our own weighted loss to combat label imbalance
def weighted_loss(y_true, y_pred):
    out = -(y_true * K.log(y_pred + 1e-5) / ratio_positive_outputs_per_device + (1.0 - y_true) * K.log(1.0 - y_pred + 1e-5))
    return K.mean(out, axis=-1)

In [9]:
def create_model(params):
    n_outputs = len(device_list)
    model = Sequential()
    model.add(LSTM(128, batch_input_shape=(params['batch_size'], None, feature_batch.shape[-1]), return_sequences=True, stateful=True))
    model.add(Dropout(params['dropout']))
    model.add(Dense(n_outputs, activation='sigmoid'))
    model.compile(loss=weighted_loss if params['use_weighted_loss'] else 'binary_crossentropy', optimizer=params['optimizer'])
    return model

training_params = {'optimizer': 'adam', 
                   'use_weighted_loss': USE_WEIGHTED_LOSS,
                   'batch_size': BATCH_SIZE,
                   'dropout': 0.0,
                   'epochs': 300}

#model = create_model(training_params)
#model.fit(train_feature_batch, train_label_batch, epochs=training_params['epochs'], batch_size=training_params['batch_size'], verbose=1, shuffle=False)

In [10]:
test_feature_batch_flattened = test_feature_batch.reshape([-1, *test_feature_batch.shape[-1:]])
test_label_batch_flattened = test_label_batch.reshape([-1, *test_label_batch.shape[-1:]])
test_feature_batch_expanded = test_feature_batch_flattened if len(test_feature_batch_flattened.shape) == 3 else np.expand_dims(test_feature_batch_flattened, axis=1)

In [11]:
def eval_model_params(params, train_X, train_Y, test_X, test_Y):
    model = create_model(training_params)
    history = model.fit(train_X, train_Y, validation_data=(test_X, test_Y), epochs=params['epochs'], batch_size=params['batch_size'], verbose=1, shuffle=False)
    return model, history.history['val_loss'][-1]


model, result = eval_model_params(training_params, train_feature_batch, train_label_batch, test_feature_batch, test_label_batch)
print(result)

Train on 32 samples, validate on 16 samples
Epoch 1/300
32/32 [==============================] - 1s 22ms/step - loss: 1.0084 - val_loss: 2.0172
Epoch 2/300
32/32 [==============================] - 0s 1ms/step - loss: 0.9287 - val_loss: 2.0010
Epoch 3/300
32/32 [==============================] - 0s 1ms/step - loss: 0.8714 - val_loss: 1.9358
Epoch 4/300
32/32 [==============================] - 0s 1ms/step - loss: 0.8195 - val_loss: 1.8256
Epoch 5/300
32/32 [==============================] - 0s 2ms/step - loss: 0.7699 - val_loss: 1.6918
Epoch 6/300
32/32 [==============================] - 0s 1ms/step - loss: 0.7253 - val_loss: 1.5620
Epoch 7/300
32/32 [==============================] - 0s 1ms/step - loss: 0.6880 - val_loss: 1.4519
Epoch 8/300
32/32 [==============================] - 0s 1ms/step - loss: 0.6581 - val_loss: 1.3650
Epoch 9/300
32/32 [==============================] - 0s 1ms/step - loss: 0.6315 - val_loss: 1.2988
Epoch 10/300
32/32 [==============================] - 0s 1ms/ste

32/32 [==============================] - 0s 909us/step - loss: 0.1627 - val_loss: 1.0998
Epoch 83/300
32/32 [==============================] - 0s 944us/step - loss: 0.1534 - val_loss: 1.1241
Epoch 84/300
32/32 [==============================] - 0s 867us/step - loss: 0.1511 - val_loss: 1.1837
Epoch 85/300
32/32 [==============================] - 0s 1ms/step - loss: 0.1525 - val_loss: 1.1823
Epoch 86/300
32/32 [==============================] - 0s 1ms/step - loss: 0.1491 - val_loss: 1.0624
Epoch 87/300
32/32 [==============================] - 0s 995us/step - loss: 0.1492 - val_loss: 0.9940
Epoch 88/300
32/32 [==============================] - 0s 977us/step - loss: 0.1530 - val_loss: 1.1114
Epoch 89/300
32/32 [==============================] - 0s 982us/step - loss: 0.1451 - val_loss: 1.1587
Epoch 90/300
32/32 [==============================] - 0s 996us/step - loss: 0.1423 - val_loss: 1.1193
Epoch 91/300
32/32 [==============================] - 0s 989us/step - loss: 0.1381 - val_loss: 1.14

32/32 [==============================] - 0s 976us/step - loss: 0.0884 - val_loss: 1.4399
Epoch 163/300
32/32 [==============================] - 0s 936us/step - loss: 0.0873 - val_loss: 1.4353
Epoch 164/300
32/32 [==============================] - 0s 903us/step - loss: 0.0951 - val_loss: 1.4821
Epoch 165/300
32/32 [==============================] - 0s 1ms/step - loss: 0.0874 - val_loss: 1.5136
Epoch 166/300
32/32 [==============================] - 0s 1ms/step - loss: 0.0863 - val_loss: 1.5503
Epoch 167/300
32/32 [==============================] - 0s 1ms/step - loss: 0.0872 - val_loss: 1.5745
Epoch 168/300
32/32 [==============================] - 0s 1ms/step - loss: 0.0913 - val_loss: 1.5627
Epoch 169/300
32/32 [==============================] - 0s 1ms/step - loss: 0.0872 - val_loss: 1.5603
Epoch 170/300
32/32 [==============================] - 0s 1ms/step - loss: 0.0857 - val_loss: 1.5938
Epoch 171/300
32/32 [==============================] - 0s 1ms/step - loss: 0.0880 - val_loss: 1.592

32/32 [==============================] - 0s 993us/step - loss: 0.0649 - val_loss: 2.0396
Epoch 243/300
32/32 [==============================] - 0s 1ms/step - loss: 0.0638 - val_loss: 2.0461
Epoch 244/300
32/32 [==============================] - 0s 1ms/step - loss: 0.0655 - val_loss: 2.0623
Epoch 245/300
32/32 [==============================] - 0s 1ms/step - loss: 0.0641 - val_loss: 2.0808
Epoch 246/300
32/32 [==============================] - 0s 1ms/step - loss: 0.0620 - val_loss: 2.0790
Epoch 247/300
32/32 [==============================] - 0s 1ms/step - loss: 0.0644 - val_loss: 2.1179
Epoch 248/300
32/32 [==============================] - 0s 1ms/step - loss: 0.0643 - val_loss: 2.1199
Epoch 249/300
32/32 [==============================] - 0s 1ms/step - loss: 0.0629 - val_loss: 2.0894
Epoch 250/300
32/32 [==============================] - 0s 1ms/step - loss: 0.0647 - val_loss: 2.0617
Epoch 251/300
32/32 [==============================] - 0s 1ms/step - loss: 0.0673 - val_loss: 2.1020
Ep

In [12]:
model.reset_states()
print(test_feature_batch_expanded.shape)

predictions = model.predict(test_feature_batch_expanded, batch_size=BATCH_SIZE)
predictions = np.squeeze(predictions)

print(predictions.shape)
print(test_label_batch_flattened.shape)

print(np.round(predictions)[:-24])
print(test_label_batch_flattened[:-24])

(320, 1, 8)
(320, 6)
(320, 6)
[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 ...
 [1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1.]]
[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 ...
 [0. 1. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]]


In [13]:
print("Val accuracy: ", np.sum(np.round(predictions) == test_label_batch_flattened) / predictions.size) 
print("Val accuracy per device:: ", np.sum(np.round(predictions) == test_label_batch_flattened, axis=0) / predictions.shape[0]) 

print("% of 1 prediction outputs", np.sum(np.round(predictions)) / predictions.size) 
print("% of 1 label outputs", np.sum(np.round(test_label_batch_flattened)) / test_label_batch_flattened.size)

Val accuracy:  0.7239583333333334
Val accuracy per device::  [0.65625  0.659375 0.7125   0.734375 0.69375  0.8875  ]
% of 1 prediction outputs 0.19270833333333334
% of 1 label outputs 0.22291666666666668


In [14]:
model.save('model.h5')

In [15]:
import keras.losses
keras.losses.weighted_loss = weighted_loss

test_model_params = dict(training_params)
test_model_params['batch_size'] = 1

#test_model = load_model("model.h5)
test_model = create_model(test_model_params)
test_model.load_weights('model.h5')

In [16]:
def test(model, in_file):
    test_features, test_labels, device_list = read_and_preprocess_data(in_file, batch_size=1)
    print(feature_batch.shape)
    print(label_batch.shape)
    predictions = np.concatenate(model.predict(feature_batch, batch_size=1), axis=0)
    print(np.round(predictions))
    print(label_batch)

In [17]:
test(test_model, in_file)

File challenge/data/device_activations_small.csv has 125 timesteps (hours)
(48, 20, 8)
(48, 20, 6)
[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]
[[[0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0.]
  [1. 0. 1. 0. 1. 0.]
  ...
  [0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0.]]

 [[1. 1. 0. 0. 0. 0.]
  [1. 1. 0. 0. 0. 0.]
  [1. 1. 0. 0. 0. 0.]
  ...
  [0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0.]]

 [[1. 1. 0. 0. 0. 0.]
  [1. 1. 0. 0. 0. 0.]
  [1. 1. 0. 0. 0. 0.]
  ...
  [0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0.]]

 ...

 [[0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0.]
  ...
  [0. 1. 0. 0. 1. 0.]
  [0. 1. 0. 0. 0. 0.]
  [0. 1. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 1. 0.]
  [0. 0. 1. 0. 1. 1.]
  ...
  [0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0.]]

 [[0. 0. 1. 1. 1. 0.]
  [0. 0. 1. 1. 1. 0.]
  [0. 0. 1. 1. 1. 0

In [18]:
def predict_next_24h(model, in_file):
    feature_batch, label_batch, device_list = read_and_preprocess_data(in_file, batch_size=1)
    print(feature_batch.shape)
    print(label_batch.shape)
    predictions = np.concatenate(model.predict(feature_batch, batch_size=1), axis=0)
    
    all_predictions = []
    
    last_features = feature_batch[-1, -1]
    last_predictions = tmp_prediction = predictions[-1]
    
    tmp_features = np.array(last_features)
    tmp_features = np.concatenate([tmp_features[:2], last_predictions])
    for i in range(24):
        print(tmp_features)
        #print(tmp_prediction)
        tmp_prediction = model.predict(np.reshape(tmp_features, [1, 1, len(tmp_features)]))
        tmp_features = np.concatenate([tmp_features[:2], tmp_prediction[0, 0]])
        
        # Increment time features
        if tmp_features[1] == 23:
            tmp_features[0] = (tmp_features[0] + 1) % 7
        tmp_features[1] = (tmp_features[1] + 1) % 24
        all_predictions += [tmp_prediction]
        
    return np.round(np.concatenate(all_predictions))

In [19]:
test_file = "challenge/data/device_activations_smaller.csv"
future_predictions = predict_next_24h(test_model, test_file)

File challenge/data/device_activations_smaller.csv has 101 timesteps (hours)
(1, 101, 8)
(1, 101, 6)
[1.         8.         0.4905757  0.52424389 0.96454465 0.92275691
 0.98310781 0.99123621]
[1.         9.         0.55714828 0.78615808 0.9591285  0.90150356
 0.98801219 0.98402739]
[ 1.         10.          0.63118595  0.93917292  0.93931311  0.77016407
  0.99010706  0.92764586]
[ 1.         11.          0.55081654  0.98028439  0.78443027  0.29767621
  0.98191595  0.46374705]
[ 1.         12.          0.21132828  0.97757304  0.17721161  0.03371891
  0.82908475  0.04795123]
[1.00000000e+00 1.30000000e+01 8.39546844e-02 8.90964866e-01
 1.30461482e-02 5.69049967e-03 1.42055795e-01 5.86819742e-03]
[1.00000000e+00 1.40000000e+01 4.88298014e-02 4.13005680e-01
 2.13941513e-03 2.51513347e-03 1.12842107e-02 2.30253744e-03]
[1.00000000e+00 1.50000000e+01 3.02042477e-02 1.21193908e-01
 9.58568940e-04 2.21730210e-03 4.23723739e-03 2.37838388e-03]
[1.00000000e+00 1.60000000e+01 1.62274484e-02 5.075

In [20]:
label_file = "challenge/data/device_activations_small.csv"
test_model.reset_states()
feature_batch, label_batch, device_list = read_and_preprocess_data(label_file, batch_size=1)
label_batch = label_batch.squeeze()
print(label_batch.shape)
print(future_predictions.shape)
future_predictions = np.squeeze(future_predictions.astype(np.int64))
print(future_predictions)
print(label_batch[-24:])
future_labels = label_batch[-24:]

File challenge/data/device_activations_small.csv has 125 timesteps (hours)
(125, 6)
(24, 1, 6)
[[1 1 1 1 1 1]
 [1 1 1 1 1 1]
 [1 1 1 0 1 0]
 [0 1 0 0 1 0]
 [0 1 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]]
[[0 0 1 1 1 0]
 [0 0 1 1 1 0]
 [0 0 1 1 1 0]
 [0 0 1 1 1 0]
 [0 1 1 1 1 0]
 [1 1 1 1 1 0]
 [1 1 1 1 1 0]
 [1 1 1 1 1 1]
 [0 1 0 0 1 0]
 [0 1 0 0 0 0]
 [0 1 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [1 0 0 0 1 0]
 [1 1 1 0 1 0]
 [1 1 1 1 1 1]
 [1 1 1 1 1 1]]


In [21]:
print(future_predictions.shape)
print(future_labels.shape)

print("Test accuracy: ", np.sum(np.round(future_predictions) == future_labels) / future_labels.size) 
print("Test accuracy per device:: ", np.sum(np.round(future_predictions) == future_labels, axis=0) / future_labels.shape[0]) 


(24, 6)
(24, 6)
Test accuracy:  0.6319444444444444
Test accuracy per device::  [0.58333333 0.45833333 0.66666667 0.66666667 0.625      0.79166667]


In [22]:
# Sandbox
n = 128
n_sequences = 4
sequence_length = 16
a = np.arange(256)
n_minibatches = n // sequence_length // n_sequences
b = sequences = np.reshape(a, [n_sequences, n_minibatches, sequence_length, 2])
print(b)
print(b.shape)
sequence_shift = n // sequence_length

mini_batch_features_arr_shape = [n_minibatches * n_sequences, sequence_length, 2]
mini_batch_features = np.zeros(mini_batch_features_arr_shape)
mini_batch_labels_arr_shape = [n_minibatches * n_sequences, sequence_length, 2]
mini_batch_labels = np.zeros(mini_batch_labels_arr_shape)
for i in range(n_minibatches):
    for j in range(n_sequences):
        mini_batch_features[i * n_sequences + j] = b[j, i]
        mini_batch_labels[i * n_sequences + j] = b[j, i]
        print(i * n_sequences + j)
        
print(mini_batch_features)

[[[[  0   1]
   [  2   3]
   [  4   5]
   [  6   7]
   [  8   9]
   [ 10  11]
   [ 12  13]
   [ 14  15]
   [ 16  17]
   [ 18  19]
   [ 20  21]
   [ 22  23]
   [ 24  25]
   [ 26  27]
   [ 28  29]
   [ 30  31]]

  [[ 32  33]
   [ 34  35]
   [ 36  37]
   [ 38  39]
   [ 40  41]
   [ 42  43]
   [ 44  45]
   [ 46  47]
   [ 48  49]
   [ 50  51]
   [ 52  53]
   [ 54  55]
   [ 56  57]
   [ 58  59]
   [ 60  61]
   [ 62  63]]]


 [[[ 64  65]
   [ 66  67]
   [ 68  69]
   [ 70  71]
   [ 72  73]
   [ 74  75]
   [ 76  77]
   [ 78  79]
   [ 80  81]
   [ 82  83]
   [ 84  85]
   [ 86  87]
   [ 88  89]
   [ 90  91]
   [ 92  93]
   [ 94  95]]

  [[ 96  97]
   [ 98  99]
   [100 101]
   [102 103]
   [104 105]
   [106 107]
   [108 109]
   [110 111]
   [112 113]
   [114 115]
   [116 117]
   [118 119]
   [120 121]
   [122 123]
   [124 125]
   [126 127]]]


 [[[128 129]
   [130 131]
   [132 133]
   [134 135]
   [136 137]
   [138 139]
   [140 141]
   [142 143]
   [144 145]
   [146 147]
   [148 149]
   [150 151]


In [23]:
indexes=[]
for x in range(mini_batch_count):
    for i in range(batch_size):
        for j in range(sequence_length):
            indexes += [i * sequence_shift + x * sequence_length + j]
print(np.reshape(indexes, [mini_batch_count, batch_size, sequence_length]))

NameError: name 'mini_batch_count' is not defined